# Data Set

Αποθηκεύω το data.set , train.set, test.set

In [1]:
# -*- coding: utf-8 -*-
from sklearn.decomposition import TruncatedSVD
from string import punctuation
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn import preprocessing
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import string
from sklearn.pipeline import Pipeline
import numpy as np

In [2]:
#apo8ikeuw ta csv arxeia sti train_set,test_set kai answers_of_test_set
train_set = pd.read_csv("../data/train.csv", encoding= 'unicode_escape')
test_set = pd.read_csv("../data/impermium_verification_set.csv", encoding= 'unicode_escape')
answers_of_test_set = pd.read_csv("../data/impermium_verification_labels.csv", encoding= 'unicode_escape')
train_set["Comment"]

0                                    "You fuck your dad."
1       "i really don't understand your point.  It see...
2       "A\xc2\xa0majority of Canadians can and has be...
3       "listen if you dont wanna get married to a man...
4       "Các bạn xuống đường biểu tình 2011 có ôn hoà ...
                              ...                        
3942    "you are both morons and that is never happening"
3943    "Many toolbars include spell check, like Yahoo...
3944    "@LambeauOrWrigley  @K.Moss \nSioux Falls, S.D...
3945    "How about Felix? He is sure turning into one ...
3946    "You're all upset, defending this hipster band...
Name: Comment, Length: 3947, dtype: object

# Προεπεξεργασία και καθάρισμα των δεδομένων

Καθαρίζω τα comments από punctuation,links, \n και ότι ξεκινάει με "\" το οποίο είναι "σκουπίδι" και δε χρειάζεται.

In [3]:
import re
from string import punctuation
#gia ta links
pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

#ka8arizei ena text sumvola, an einai adeio to kanei null
def clean_string(text): # ta kanei null
    #text = " ".join(filter(lambda x:x[0]!="\\", text.split()))
    text = pattern.sub("",text)
    text = text.replace('\n','')
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    if text == '': 
        text = 'NULL' 
    return text

In [4]:
#remove punctuation from train_set and test set
for i in range(0,len(train_set["Comment"])):
    temp = clean_string(train_set["Comment"][i])
    train_set["Comment"].replace(to_replace=train_set["Comment"][i],value=temp,inplace=True)

for i in range(0,len(test_set["Comment"])):
    temp = clean_string(test_set["Comment"][i])
    test_set["Comment"].replace(to_replace=test_set["Comment"][i],value=temp,inplace=True)
    
for i in range(0,len(answers_of_test_set["Comment"])):
    temp = clean_string(answers_of_test_set["Comment"][i])
    answers_of_test_set["Comment"].replace(to_replace=answers_of_test_set["Comment"][i],value=temp,inplace=True)
    

answers_of_test_set['Comment']

0                        like this if you are a tribe fan
1                                             youre idiot
2       i am a woman babs and the only war on women i ...
3       wow  you benefitted so many wins this year fro...
4       haha green me red you now loser whos winning n...
                              ...                        
2230    fuckin lame come on wtf stop fucking over my b...
2231    you shut your ignorant pie hole you little ins...
2232    sweetie pie is looking very much like her cous...
2233      ball4real where are you with your miami gayness
2234    manif you are a 3 point shooter you must love ...
Name: Comment, Length: 2235, dtype: object

# Naive Bayes 

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import f1_score,accuracy_score


X_train = train_set["Comment"]
le = preprocessing.LabelEncoder()
le.fit(train_set["Insult"])
y_train = le.transform(train_set["Insult"])
X_test = answers_of_test_set["Comment"]
y_test = answers_of_test_set["Insult"]

#Pipelines definition
pipeline = Pipeline([ ('vector', CountVectorizer()),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',GaussianNB())])

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for GaussianNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.5230347713699546 
Accuracy: 0.5230425055928412


# Βελτιστοποίηση Naive Bayes

# 1)Stemming με τη χρήση του PorterStemmer της gensim

In [6]:
from gensim.parsing.porter import PorterStemmer
#Stemming
X_train_stemmed = PorterStemmer().stem_documents(train_set["Comment"])
X_test_stemmed = PorterStemmer().stem_documents(answers_of_test_set["Comment"])

#Pipelines definition
pipeline = Pipeline([ ('vector', CountVectorizer()),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',GaussianNB())])

y_train = train_set["Insult"]
y_test = answers_of_test_set["Insult"]

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train_stemmed,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test_stemmed)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for GaussianNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.5067096120396462 
Accuracy: 0.5082774049217003


# 2) Χρήση english stopwords και κάποιων κοινών ουδέτερων λέξεων που παρατήρησα ότι επαναλαμβάνονται συχνά σε κείμενα

In [7]:
#stopwords
from nltk.corpus import stopwords


my_stopwords = set(stopwords.words('english'))
other_words = {'said','say','says','just','did','was','were','year','years','like','people','tell','tells'
               'told','report','reported','get','also','however','already','may','almost','hes','ask','would',
              'could','one','see','dont','wanna','seems','what','that','see','all','this'}
my_stopwords = my_stopwords.union(other_words)


#Pipelines definition
pipeline = Pipeline([ ('vector', CountVectorizer(stop_words = my_stopwords)),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',GaussianNB())])

X_train = train_set["Comment"]
X_test = answers_of_test_set["Comment"]
y_train = train_set["Insult"]
y_test = answers_of_test_set["Insult"]

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for GaussianNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.5230378268953886 
Accuracy: 0.5230425055928412


# 3) Δημιουργώ τα bigrams με βαση τη παράμετρο ngram_range = (2,2)

βλέπε https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html ,
αν βάλεις (2,2) τότε δέχεται μόνο bigram σύμφωνα με τις οδηγίες της

In [8]:
#Pipelines definition
pipeline = Pipeline([ ('vector', CountVectorizer(ngram_range=(2, 2))),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',GaussianNB())])

X_train = train_set["Comment"]
X_test = answers_of_test_set["Comment"]
y_train = train_set["Insult"]
y_test = answers_of_test_set["Insult"]

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for GaussianNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.5592948508571052 
Accuracy: 0.5610738255033557


# 4) Laplace Smoothing

Το Laplace Smoothing γίνεται στη MultinomialNB όταν το alpha = 1.

In [9]:
from sklearn.naive_bayes import MultinomialNB

#Pipelines definition
pipeline = Pipeline([ ('vector', CountVectorizer()),('tfidf', TfidfTransformer()),
                     ('clf',MultinomialNB(alpha = 1))])

X_train = train_set["Comment"]
X_test = answers_of_test_set["Comment"]
y_train = train_set["Insult"]
y_test = answers_of_test_set["Insult"]

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for MultinomialNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.4041369800045967 
Accuracy: 0.5458612975391499


# Αποτελέσματα

Με βάση τα αποτελέσματα φαίνεται ότι η μόνη μέθοδος που βοήθησε λίγο ήταν τα bigrams, τα stopwords δεν άλλαξαν τίποτα και τα υπόλοιπα 2 απλά έκανα ακόμα χειρότερο το αποτέλεσμα. Η MultinomialNB θα εκτελούνταν καλύτερα αν δε χρησιμοποιούσαμε Laplace smoothing.

# Part-of-Speech Based Features

In [10]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag,pos_tag_sents
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from collections import Counter

In [11]:
le = preprocessing.LabelEncoder()
le.fit(train_set["Insult"])
y_train = le.transform(train_set["Insult"])
y_test = answers_of_test_set["Insult"]

# POS features

Χρησιμοποιώ τη pos_tag_sents για να ταξινομήσω τις λέξεις και κρατάω μόνο εκείνες που είναι NOUN, VERB, ADV, ADJ
και το total που είναι το σύνολο των λέξεων που ανήκουν σε αυτές τις κατηγορίες. Το total θα χρειασθεί στη συνέχεια
για τα frequency τους.

In [12]:
X_train = []
X_test = []
total1 = 0 #sunolo le3ewn pou anikoun stis tesseris autes katigories
total2 = 0
train = pos_tag_sents( train_set['Comment'].apply(word_tokenize),tagset='universal')
for entry in train:
    final_result = []
    for word in entry:
        if (word[1] == 'NOUN' or word[1] == 'VERB' or word[1] == 'ADV' or word[1] == 'ADJ'):
            total1 += 1
            final_result.append(word)
    X_train.append(final_result)
    
test = pos_tag_sents( test_set['Comment'].apply(word_tokenize),tagset='universal')
for entry in test:
    final_result = []
    for word in entry:
        if (word[1] == 'NOUN' or word[1] == 'VERB' or word[1] == 'ADV' or word[1] == 'ADJ'):
            total2 += 1
            final_result.append(word)
    X_test.append(final_result)
    
print(X_train)

[[('fuck', 'VERB'), ('dad', 'NOUN')], [('i', 'NOUN'), ('really', 'ADV'), ('dont', 'ADJ'), ('understand', 'VERB'), ('point', 'NOUN'), ('seems', 'VERB'), ('are', 'VERB'), ('mixing', 'VERB'), ('apples', 'NOUN'), ('oranges', 'NOUN')], [('axc2xa0majority', 'NOUN'), ('canadians', 'NOUN'), ('can', 'VERB'), ('has', 'VERB'), ('been', 'VERB'), ('wrong', 'ADJ'), ('before', 'ADV'), ('now', 'ADV'), ('will', 'VERB'), ('be', 'VERB'), ('againnnunless', 'ADJ'), ('youre', 'ADJ'), ('supportive', 'NOUN'), ('idea', 'NOUN'), ('nothing', 'NOUN'), ('is', 'VERB'), ('full', 'ADJ'), ('proof', 'NOUN'), ('perfect', 'VERB'), ('take', 'VERB'), ('chances', 'NOUN'), ('should', 'VERB'), ('inadvertently', 'ADV'), ('kill', 'VERB'), ('son', 'NOUN'), ('daughter', 'NOUN'), ('then', 'ADV'), ('thems', 'VERB'), ('breaks', 'NOUN'), ('can', 'VERB'), ('always', 'ADV'), ('regard', 'VERB'), ('collateral', 'ADJ'), ('damage', 'NOUN'), ('wartime', 'NOUN'), ('sorry', 'VERB'), ('butxc2xa0the', 'NOUN'), ('cheques', 'NOUN'), ('mail', 'NOU

# Βρίσκω το frequency των NOUN,VERB,ADJ,ADV και τα βάζω σε μία λίστα

In [13]:
train_counts = []
for tags in X_train:
    cnt = Counter()
    for tag in tags:
        cnt[tag[1]] += 1/total1
    train_counts.append(cnt.most_common())
    
test_counts = []
for tags in X_test:
    cnt = Counter()
    for tag in tags:
        cnt[tag[1]] += 1/total2
    test_counts.append(cnt.most_common())
    
print(train_counts)

[[('VERB', 1.2287276525158199e-05), ('NOUN', 1.2287276525158199e-05)], [('NOUN', 4.9149106100632796e-05), ('VERB', 4.9149106100632796e-05), ('ADV', 1.2287276525158199e-05), ('ADJ', 1.2287276525158199e-05)], [('NOUN', 0.00018430914787737299), ('VERB', 0.00017202187135221479), ('ADJ', 6.1436382625791e-05), ('ADV', 6.1436382625791e-05)], [('VERB', 0.00022117097745284758), ('NOUN', 8.601093567610739e-05), ('ADJ', 6.1436382625791e-05), ('ADV', 2.4574553050316398e-05)], [('NOUN', 0.0008723966332862307), ('ADJ', 4.9149106100632796e-05)], [('ADJ', 9.829821220126559e-05), ('NOUN', 9.829821220126559e-05), ('VERB', 9.829821220126559e-05), ('ADV', 1.2287276525158199e-05)], [('ADV', 2.4574553050316398e-05), ('NOUN', 1.2287276525158199e-05), ('VERB', 1.2287276525158199e-05)], [('VERB', 4.9149106100632796e-05), ('NOUN', 4.9149106100632796e-05), ('ADJ', 1.2287276525158199e-05)], [('ADJ', 2.4574553050316398e-05), ('VERB', 1.2287276525158199e-05), ('ADV', 1.2287276525158199e-05)], [('VERB', 3.6861829575

# Μετατρέπω τη λίστα σε λεξικό για να τη περάσω από τον DictVectorizer και έπειτα από τον TfidfTransformer

In [14]:
final_train = []
for count in train_counts:
    info = {}
    for fraction in count:
        info[fraction[0].lower()] = fraction[1]
        #print(info)
    final_train.append(info)

final_test = []
for count in test_counts:
    info = {}
    for fraction in count:
        info[fraction[0].lower] = fraction[1]
        #print(info)
    final_test.append(info)
    
print(final_train)

[{'verb': 1.2287276525158199e-05, 'noun': 1.2287276525158199e-05}, {'noun': 4.9149106100632796e-05, 'verb': 4.9149106100632796e-05, 'adv': 1.2287276525158199e-05, 'adj': 1.2287276525158199e-05}, {'noun': 0.00018430914787737299, 'verb': 0.00017202187135221479, 'adj': 6.1436382625791e-05, 'adv': 6.1436382625791e-05}, {'verb': 0.00022117097745284758, 'noun': 8.601093567610739e-05, 'adj': 6.1436382625791e-05, 'adv': 2.4574553050316398e-05}, {'noun': 0.0008723966332862307, 'adj': 4.9149106100632796e-05}, {'adj': 9.829821220126559e-05, 'noun': 9.829821220126559e-05, 'verb': 9.829821220126559e-05, 'adv': 1.2287276525158199e-05}, {'adv': 2.4574553050316398e-05, 'noun': 1.2287276525158199e-05, 'verb': 1.2287276525158199e-05}, {'verb': 4.9149106100632796e-05, 'noun': 4.9149106100632796e-05, 'adj': 1.2287276525158199e-05}, {'adj': 2.4574553050316398e-05, 'verb': 1.2287276525158199e-05, 'adv': 1.2287276525158199e-05}, {'verb': 3.68618295754746e-05, 'noun': 3.68618295754746e-05}, {'noun': 6.1436382

# Εκτελώ τους αλγόριθμους GaussianNB,Random Forest και SVM

In [15]:
#Pipelines definition
pipeline = Pipeline([ ('vector', DictVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',GaussianNB())])

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(final_train,y_train)

#Predict the test split
y_pred = pipeline.predict(final_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for GaussianNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.34129089301503096 
Accuracy: 0.5181208053691275


In [16]:
pipeline = Pipeline([ ('vector', DictVectorizer()),('tfidf', TfidfTransformer()),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',RandomForestClassifier())])

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(final_train,y_train)

#Predict the test split
y_pred = pipeline.predict(final_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for Random Forest(RF):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for Random Forest(RF):
F-Measure: 0.34129089301503096 
Accuracy: 0.5181208053691275


In [17]:
pipeline = Pipeline([ ('vector', DictVectorizer()),('tfidf', TfidfTransformer()),
                     ('to_dense',FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
                     ('clf',svm.SVC(kernel='rbf'))])

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(final_train,y_train)

#Predict the test split
y_pred = pipeline.predict(final_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for Support Vector Machines (SVM):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for Support Vector Machines (SVM):
F-Measure: 0.34129089301503096 
Accuracy: 0.5181208053691275


# Trying to beat the Benchmark(miserably)

In [18]:
from sklearn.naive_bayes import MultinomialNB

my_stopwords = set(stopwords.words('english'))
other_words = {'said','say','says','just','did','was','were','year','years','like','people','tell','tells'
               'told','report','reported','get','also','however','already','may','almost','hes','ask','would',
              'could','one','see','dont','wanna','seems','what','that','see','all','this'}
my_stopwords = my_stopwords.union(other_words)

#Pipelines definition
pipeline = Pipeline([ ('vector', CountVectorizer(strip_accents = 'ascii')),
                     ('tfidf', TfidfTransformer()),
                     ('clf',MultinomialNB(alpha = 0.01))])

X_train = train_set["Comment"]
X_test = answers_of_test_set["Comment"]
y_train = train_set["Insult"]
y_test = answers_of_test_set["Insult"]

#"taizw" to pipeline me ta x_train,y_train
pipeline.fit(X_train,y_train)

#Predict the test split
y_pred = pipeline.predict(X_test)

f1 = f1_score(y_test,y_pred,average = 'macro')
accuracy = accuracy_score(y_test, y_pred)

print('Cross Validation results for GaussianNB(NB):'),
print('F-Measure:',f1,'\nAccuracy:',accuracy)

Cross Validation results for GaussianNB(NB):
F-Measure: 0.6400764755870603 
Accuracy: 0.6639821029082774


# Δοκιμές
Δοκίμασα και τις 4 μεθόδους που κάναμε σε προηγούμενο ερώτημα, ανεξάρτητα αλλά και σε συνδυασμούς όμως δεν έβγαλα κάποια άκρη. Δοκίμασα και αρκετά stopwords όμως πάλι δε μπορούσα να ανεβάσω το score.